# TODO：概要

# ライブラリのインポート

In [ ]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
)


# 環境変数の取得

In [2]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [3]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# ユーティリティ関数

In [141]:
def agent_run_outputs(thread_id, project_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = project_client.agents.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                project_client.agents.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

In [154]:
def file_search_content_outputs(project_client, thread_id, run_id):
    """
    指定したスレッドID・Run IDのファイル検索ツールによる検索結果コンテンツをNotebook等に出力します。

    :param project_client: Azure AI Project クライアント
    :param thread_id: スレッドID
    :param run_id: Run ID
    """
    from azure.ai.agents.models import RunAdditionalFieldList, RunStepToolCallDetails, RunStepFileSearchToolCall

    for run_step in project_client.agents.run_steps.list(
        thread_id=thread_id,
        run_id=run_id,
        include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
    ):
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and getattr(tool_call, "file_search", None)
                    and getattr(tool_call.file_search, "results", None)
                    and tool_call.file_search.results
                    and getattr(tool_call.file_search.results[0], "content", None)
                    and tool_call.file_search.results[0].content
                    and getattr(tool_call.file_search.results[0].content[0], "text", None)
                ):
                    file_name = tool_call.file_search.results[0].file_name
                    content_text = tool_call.file_search.results[0].content[0].text
                    print(f"ファイル検索の結果、以下のコンテンツを見つけました。\n「{file_name}」:")
                    print("=" * 60)
                    print(content_text)
                    print("=" * 60)

# ツールの定義

## ベクトルストアの作成

In [ ]:
# アップロード対象ディレクトリ
input_dir = "./input_data"

# ファイルIDのリスト
file_ids = []

# ディレクトリ内の全ファイルをループしてアップロード
for fname in os.listdir(input_dir):
    file_path = os.path.join(input_dir, fname)
    if os.path.isfile(file_path):
        uploaded_file = project_client.agents.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file: {fname}, file ID: {uploaded_file.id}")
        file_ids.append(uploaded_file.id)

if not file_ids:
    raise RuntimeError("No files were uploaded. Please check the directory.")

# アップロードした全ファイルを使ってベクトルストアを作成
vector_store = project_client.agents.vector_stores.create_and_poll(file_ids=file_ids, name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")


Uploaded file: Xbox_GamePassPlan.pdf, file ID: assistant-2JbwdZLqFjFz7bagjSJzKi
Uploaded file: カスタマーサービスのパンフレット.pptx, file ID: assistant-StGfi8AvM3rfYtXHLwz2de
Uploaded file: 出荷日ポリシー.md, file ID: assistant-7jDwApVC7wPzvJH51tjems
Created vector store, vector store ID: vs_ByChEiaUsjVUUQBKVHAykEWw


In [ ]:
# インポートしたファイル数の確認
vector_store.as_dict()
vector_store.file_counts

{'in_progress': 0, 'completed': 3, 'failed': 0, 'cancelled': 0, 'total': 3}

## File Search ツールの定義

In [ ]:
# Code Interpreter ツールの定義
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

# エージェントの作成

In [132]:
file_search_agent  = project_client.agents.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="file_search_agent ",
    instructions=(
        "アップロードしたファイルから情報を検索できるエージェントです。"
        "ユーザーの質問に、アップロードしたファイルを参照して答えてください。"
    ),
    description=(
        "アップロードしたファイルから情報を検索できるエージェントです。"    
    ),
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created Agent. AGENT_ID: {file_search_agent.id}")


Created Agent. AGENT_ID: asst_wzl4CnDhNG4SGCopnfVoiWG1


# スレッドの作成

In [144]:
# Thread の作成
thread = project_client.agents.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_2iVCubFtyGneFTWLnESJS722


# ユーザーメッセージの追加

In [ ]:
# メッセージの追加
user_message_1 = "ゲームをしたくて、新しいパソコンを探してます。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_1,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_LfsJDig0dbwAzUtNzLpMK553


# Run の実行

## 1 ターン目

In [ ]:
run_1 = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_1.status == "failed":
    print(f"Run failed: {run_1.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

USER: ゲームをしたくて、新しいパソコンを探してます。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: アップロードされたファイルの中で「初期設定サポート」に関する情報を検索します。少々お待ちください。
ASSISTANT: 「カスタマーサービスのパンフレット」に、初心者向けのパソコン初期設定サポートについて詳しい案内がありました。サービス内容は次の通りです：

- 所要時間：約30〜60分
- サポート内容：
  - 電源の接続と起動確認
  - Wi-Fiやインターネット接続設定
  - メールアカウントやアプリの初期登録
  - セキュリティ対策（ウイルス対策ソフトの導入など）
  - 基本操作（マウス、キーボード、タッチ操作）
  - よく使うアプリのインストールサポート（Office・Zoomなど）

また、出張対応かオンライン対応が選べて、現在は「初回サポート無料」のキャンペーン中です。希望者にはマニュアルのプレゼントもあります。

問い合わせや予約は、下記まで可能です：
- 電話：0120-123-456（平日 10:00〜18:00）
- メール：support@easysetup.jp
- WEB予約：https://www.easysetup.jp

このサポートは「初めてパソコンやスマホを買う方」や「設定に不安のある方」に特におすすめです【5:0†カスタマーサービスのパンフレット.pptx】。

ご希望の場合、上記連絡先からご相談・ご予約ください。


In [ ]:
# 参照したコンテンツを出力
file_search_content_outputs(project_client, thread.id, run_1.id)


ファイル検索の結果、以下のコンテンツを見つけました。
「カスタマーサービスのパンフレット.pptx」:
🎉初心者向けセットアップサポート🎉 ～パソコン・スマホの初期設定を安心サポート～ 
✅ サービス内容（所要時間：約30〜60分） 電源の接続と起動確認 Wi-Fi・インターネット接続設定 メールアカウント・アプリの初期登録 セキュリティ対策（ウイルス対策ソフトの確認・導入） 基本操作レクチャー（マウス、キーボード、タッチ操作） よく使うアプリのインストールサポート（ Office ・Zoomなど） 
👩‍💻 こんな方におすすめ ！ 初めてパソコンやスマホを購入した方 設定がよくわからず不安な方 家族に頼らず自分で使いたい方 
📞 カスタマーサポート ご不明点やご予約は下記までお気軽に！ 電話：0120-123-456（平日 10:00〜18:00） メール：support@easysetup.jp WEB予約： https://www.easysetup.jp 
💡安心のサポート体制 ✔ 出張対応／オンライン対応 選べます ✔ 初回サポート無料（キャンペーン中） ✔ ご希望に応じてマニュアルもプレゼント！


## 2 ターン目

In [155]:
# メッセージの追加
user_message_2 = "一番安い Xbox ゲームパスプランについて教えてください。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_2,
)

run_2 = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_2.status == "failed":
    print(f"Run failed: {run_2.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

USER: ゲームをしたくて、新しいパソコンを探してます。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: アップロードされたファイルの中で「初期設定サポート」に関する情報を検索します。少々お待ちください。
ASSISTANT: 「カスタマーサービスのパンフレット」に、初心者向けのパソコン初期設定サポートについて詳しい案内がありました。サービス内容は次の通りです：

- 所要時間：約30〜60分
- サポート内容：
  - 電源の接続と起動確認
  - Wi-Fiやインターネット接続設定
  - メールアカウントやアプリの初期登録
  - セキュリティ対策（ウイルス対策ソフトの導入など）
  - 基本操作（マウス、キーボード、タッチ操作）
  - よく使うアプリのインストールサポート（Office・Zoomなど）

また、出張対応かオンライン対応が選べて、現在は「初回サポート無料」のキャンペーン中です。希望者にはマニュアルのプレゼントもあります。

問い合わせや予約は、下記まで可能です：
- 電話：0120-123-456（平日 10:00〜18:00）
- メール：support@easysetup.jp
- WEB予約：https://www.easysetup.jp

このサポートは「初めてパソコンやスマホを買う方」や「設定に不安のある方」に特におすすめです【5:0†カスタマーサービスのパンフレット.pptx】。

ご希望の場合、上記連絡先からご相談・ご予約ください。
USER: 一番安い Xbox ゲームパスプランについて教えてください。
ASSISTANT: アップロードされた「Xbox_GamePassPlan.pdf」によると、一番安いXboxゲームパスのプランは「Xbox Game Pass Core」です。

このプランのポイントは以下のとおりです：

- 月額料金：924円（税込）
- サービス内容：
  - オンラインマルチプレイ
  - 一部タイトルのセレクトカタログ（約25タイトル前後）
  - 会員限定特典や割引あり

特に「すぐにとにかく安くXboxでオンラインを楽しみたい」方や「初めてXboxを利用する方」に人気のプランです。

他の上位プラン（Ultimateなど）と比べて、利用できる

In [156]:
# 参照したコンテンツを出力
file_search_content_outputs(project_client, thread.id, run_2.id)

ファイル検索の結果、以下のコンテンツを見つけました。
「カスタマーサービスのパンフレット.pptx」:
🎉初心者向けセットアップサポート🎉 ～パソコン・スマホの初期設定を安心サポート～ 
✅ サービス内容（所要時間：約30〜60分） 電源の接続と起動確認 Wi-Fi・インターネット接続設定 メールアカウント・アプリの初期登録 セキュリティ対策（ウイルス対策ソフトの確認・導入） 基本操作レクチャー（マウス、キーボード、タッチ操作） よく使うアプリのインストールサポート（ Office ・Zoomなど） 
👩‍💻 こんな方におすすめ ！ 初めてパソコンやスマホを購入した方 設定がよくわからず不安な方 家族に頼らず自分で使いたい方 
📞 カスタマーサポート ご不明点やご予約は下記までお気軽に！ 電話：0120-123-456（平日 10:00〜18:00） メール：support@easysetup.jp WEB予約： https://www.easysetup.jp 
💡安心のサポート体制 ✔ 出張対応／オンライン対応 選べます ✔ 初回サポート無料（キャンペーン中） ✔ ご希望に応じてマニュアルもプレゼント！


## 3 ターン目

In [ ]:
# メッセージの追加
user_message_3 = "ゲームをしたくて、新しいパソコンを探してます。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_3,
)

run_3 = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_3.status == "failed":
    print(f"Run failed: {run_3.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

In [ ]:
# 参照したコンテンツを出力
file_search_content_outputs(project_client, thread.id, run_3.id)